#### Table Content
* SQL DB
* Extract Frame from Video
* Extract Frame from RTPS
* Log Frame metadata into SQL DB

In [1]:
import mysql.connector

conn = mysql.connector.connect(host="127.0.0.1", port=3306,
                               user="root", password="qwerty")

class SQLHelper:

    def __init__(self, dbname, table_name):
        self.dbname = dbname
        self.table_name = table_name
        self.conn = mysql.connector.connect(host="127.0.0.1", port=3306,
                               user="root", password="qwerty", database = self.dbname)
        
    def GetTablesData(self, table_name):
        
        # Create connecation
        cur = self.conn.cursor()

        # Use Database
        cur.execute(f"USE {self.dbname}")
        
        # Execute a query
        cur.execute(f"SELECT * FROM {table_name}")
        
        # Fetch one result
        row = cur.fetchall()
        
        return row

#         print([db for db in row])

    def RunQuery(self, query, verbose=True):
        # Create a cursor
        cur = self.conn.cursor()

        # Execute the query
        cur.execute(query)

        # Commit the changes (assuming you are making changes to the database)
        self.conn.commit()

        if verbose:
            print("Query Executed")

    def ShowTables(self, dbname):

        # Create connecation
        cur = conn.cursor()

        # Use Database
        cur.execute(f"USE {dbname}")
        
        # Execute a query
        cur.execute("SHOW TABLES")
        
        # Fetch one result
        row = cur.fetchall()

        print([db for db in row])


    def ShowDatabase(self):
        cur = conn.cursor()
        
        # Execute a query
        cur.execute("SHOW DATABASES")
        
        # Fetch one result
        row = cur.fetchall()

        print([db for db in row])

    def __get_datetime__(self):
        # Get a cursor
        cur = conn.cursor()
        
        # Execute a query
        cur.execute("SELECT current_timestamp()")
        
        # Fetch one result
        row = cur.fetchone()
        
        print("Current date is: {0}".format(row[0]))
        
        # Close connection
        # conn.close()

In [2]:
dbname, table_name = 'RTPSDB', 'FrameTable'

sql_obj = SQLHelper(dbname, table_name)

In [3]:
sql_obj.ShowTables(dbname)

[('extractedframefromvideo',)]


In [4]:
data = sql_obj.GetTablesData("extractedframefromvideo")

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(data, columns=[['frame_id', 'Cam_id', 'date_time', 'Site_id','username', 'email']])
df.head()

,frame_id,Cam_id,date_time,Site_id,username,email
0,121620232331,1,2023-12-16 23:19:21,1,tarunchaubey,tarunchaubey09@gmail.com
1,131620232331,1,2023-12-16 23:19:21,1,tarunchaubey,tarunchaubey09@gmail.com
2,1316720232331,1,2023-12-16 23:19:21,1,tarunchaubey,tarunchaubey09@gmail.com
3,1217202316304496,1,2023-12-16 23:19:21,1,tarunchaubey,tarunchaubey09@gmail.com
4,1217202317041617,1,2023-12-16 23:19:21,1,tarunchaubey,tarunchaubey09@gmail.com


In [7]:
df.tail()

,frame_id,Cam_id,date_time,Site_id,username,email
32377,122020232343593292,1,2023-12-20 23:43:59.329272,47,tarunchaubey,tarunchaubey09@gmail.com
32378,122020232343593661,2,2023-12-20 23:43:59.366842,91,tarunchaubey,tarunchaubey09@gmail.com
32379,122020232343593731,1,2023-12-20 23:43:59.373127,17,tarunchaubey,tarunchaubey09@gmail.com
32380,122020232343593988,1,2023-12-20 23:43:59.398817,89,tarunchaubey,tarunchaubey09@gmail.com
32381,122020232343594077,2,2023-12-20 23:43:59.407754,92,tarunchaubey,tarunchaubey09@gmail.com


In [8]:
df.shape

(32382, 6)

In [9]:
df.dtypes

frame_id              int64
Cam_id                int64
date_time    datetime64[ns]
Site_id               int64
username             object
email                object
dtype: object

In [7]:
from threading import BoundedSemaphore,Lock
IsLocok = Lock()

In [9]:
import cv2
import uuid
from datetime import datetime
from threading import Thread
import numpy as np
from threading import BoundedSemaphore, Lock

IsLocok = Lock()


dbname, table_name = 'RTPSDB', 'FrameTable'

sql_obj = SQLHelper(dbname, table_name)


class StreamVideoPreprocessing:
    
    @staticmethod
    def ExtractFreame(source_link, cam_id):
        
        cap = cv2.VideoCapture(source_link)

        count = cap.get(cv2.CAP_PROP_FPS)

        while cap.isOpened():

            ret, frame = cap.read()

            if ret is None:
                print("File Not Read")

            
            if count%30==0:
            
                IsLocok.acquire()

                frame_id = datetime.now().strftime("%m%d%Y%H%M%S%f")[:-2]

                date_time = str(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"))
                
                frame = cv2.putText(frame, f"{date_time}", (40, 40), cv2.FONT_ITALIC, 0.7, (0, 0, 255), 1)
                
                data = {"frame_id": frame_id, "cam_id": cam_id, "date_time": date_time,
                    "Site_id": np.random.randint(0,100,1)[0],"username": "tarunchaubey","email_id": "tarunchaubey09@gmail.com"}

                query = f"""INSERT INTO ExtractedFrameFromVideo (frame_id, cam_id, date_time, Site_id, username, email_id) VALUES {tuple(data.values())};"""
                
                sql_obj.RunQuery(query, verbose=False)
            
                IsLocok.release()

                print(data,"\n")
            
            count+=1

            # cv2.imshow("VideoApps", frame)

            # Use a small delay (e.g., 1 millisecond)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()


# Test source_link_a
# t1 = Thread(target=StreamVideoPreprocessing.ExtractFreame, args=(0, 1))
t1 = Thread(target=StreamVideoPreprocessing.ExtractFreame, args=(0, 1))
t1.start()

# Test source_link_b
t2 = Thread(target=StreamVideoPreprocessing.ExtractFreame, args=("http://192.168.1.6:4747/video", 2))
t2.start()

# t1.join()

# t2.join()

{'frame_id': '122020232350592728', 'cam_id': 1, 'date_time': '2023-12-20 23:50:59.272891', 'Site_id': 1, 'username': 'tarunchaubey', 'email_id': 'tarunchaubey09@gmail.com'} 



{'frame_id': '122020232350595806', 'cam_id': 2, 'date_time': '2023-12-20 23:50:59.580646', 'Site_id': 26, 'username': 'tarunchaubey', 'email_id': 'tarunchaubey09@gmail.com'} 

{'frame_id': '122020232351002967', 'cam_id': 1, 'date_time': '2023-12-20 23:51:00.296704', 'Site_id': 78, 'username': 'tarunchaubey', 'email_id': 'tarunchaubey09@gmail.com'} 

{'frame_id': '122020232351010835', 'cam_id': 2, 'date_time': '2023-12-20 23:51:01.083593', 'Site_id': 61, 'username': 'tarunchaubey', 'email_id': 'tarunchaubey09@gmail.com'} 

{'frame_id': '122020232351012573', 'cam_id': 1, 'date_time': '2023-12-20 23:51:01.257390', 'Site_id': 38, 'username': 'tarunchaubey', 'email_id': 'tarunchaubey09@gmail.com'} 

{'frame_id': '122020232351020684', 'cam_id': 1, 'date_time': '2023-12-20 23:51:02.068400', 'Site_id': 3, 'username': 'tarunchaubey', 'email_id': 'tarunchaubey09@gmail.com'} 

{'frame_id': '122020232351020897', 'cam_id': 1, 'date_time': '2023-12-20 23:51:02.089737', 'Site_id': 86, 'username': 'ta